In [ ]:
import verst_analytics as va
import pandas
import sklearn.tree
import graphviz

In [ ]:
query = """
select partner_api_id, event_name, count(1) cnt from kit.events group by 1, 2
"""

activity = va.query_rs(query).set_index('partner_api_id').pivot(columns='event_name', values='cnt')

In [ ]:
query = """
select api_id, liveness from sites
"""

status = va.query_pg(query).set_index('api_id')

LIVENESS_MAPPING_STATUS = {
    0: 'error',
    1: 'error',
    2: 'expired',
    3: 'trial',
    4: 'error',
    5: 'error',
    6: 'error',
    7: 'paying'
}

LIVENESS_MAPPING_STATUS = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 1
}

status['liveness'] = status['liveness'].apply(lambda x: LIVENESS_MAPPING_STATUS[x])

In [ ]:
df = activity.join(status,how='outer').fillna(0)

In [ ]:
y = df['liveness'].values
X = df.drop(['liveness',
             'publisher:plan:checkout:card:button:click',
             'publisher:plan:confirmation:-:-:impression',
             'publisher:plan:checkout:coupon_code:button:click',
             'publisher:plan:checkout:-:-:impression'], axis=1)
dt = sklearn.tree.DecisionTreeClassifier(max_depth=3)
dt.fit(X, y)

In [ ]:
dot_data = sklearn.tree.export_graphviz(dt, 
                                        out_file='temp.dot', 
                                        feature_names=X.columns,
                                        proportion=True)
with open("temp.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)